In [2]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
#from spellchecker import SpellChecker
#import matplotlib.pyplot as plt
from collections import Counter #uso para n-gramas
#from wordcloud import WordCloud
import re # talvez regex vai ajudar na normalizaçao

Aqui pretendo verificar a frequência das palavras (devo retirar stopwords) para fazer um heatmap.
Desafio extra: subir o texto de insumo, fazer o mesmo pré-processamento e exibir somente heywords que não aparecem no texto de insumo.

In [13]:
csv_file_path = 'corpus_completo.csv'

# Lendo o csv como um df
df = pd.read_csv(csv_file_path)

#Criando uma cópia
df_va = df.copy()

In [14]:
df_va.groupby('nota').count()

,file_name,content
nota,,
0,25,25
1,211,211
2,628,628
3,715,715
4,477,477
5,237,237


In [15]:
nltk.download('stopwords')
nltk.download('punkt') # é um tokenizador, importante para nltk.word_tokenize
# nltk.download('rslp') é um stemmer. acho q nao vou usar
nltk.download('punkt_tab') # Precisei que devia fazer download desse pacote pq o punkt nao tava funcionando

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [16]:
# Pré-processamento

# Devo tirar acentos?? Ex: Belém
# Retirar números? Na duvida, retirei.
# Não fiz stemming visto que poderia perder erros ortográficos

#Retirada de sinais gráficos, pontuações e espaços
def clean_cp(text):
    cleaned = text.lower() #Deixando tudo minúsculo
    cleaned = re.sub('[^\w\s]', '', cleaned) # Removendo pontuacao
    #cleaned = re.sub('[0-9]+', '', cleaned) # Removendo números 
    cleaned = re.sub('\d+', '', cleaned) # Removendo números NÃO TÁ FUNCIONANDO AAAAAAAAAAAAAAAAA Começou a funcionar qdo pus o lower como primeiro comando da funçao
    cleaned = re.sub('\s+', ' ', cleaned) # Removendo espaços extras
    cleaned = re.sub('\s+', ' ', cleaned)
    return cleaned.strip() # Removendo tabs

#Retirada de stopwords.. Essa parte deve ser importante na hora de ver as palavras mais frequentes (heatmap)
#def stopwords_cp(text):
  #  stopwords = nltk.corpus.stopwords.words('portuguese')
   # tokenized = nltk.word_tokenize(text, language='portuguese')
   # sem_stopwords = [token for token in tokenized if token not in stopwords]
   # return ' '.join(sem_stopwords)

df_va['content'] = df_va['content'].apply(clean_cp)
#df_va['content'] = df_va['content'].apply(stopwords_cp)

In [17]:
df_va['content'].sample(5)

2224    prezada prefeitura municipal sou morador de be...
2146    belém do para de outubro de senhores prefeitur...
1302    belém do pará de outubro de prefeitura municip...
1371    belém julho de carta aberta senhores quem se i...
1942    caro prefeito municipal belém tem sido conside...
Name: content, dtype: object

In [18]:
# Tokenizando e retirando stopwords: importante para ver a frequencia das palavras

def tokenized_cp(text):
   stopwords = nltk.corpus.stopwords.words('portuguese') # Carregando as stopwords do português
   tokenized = nltk.word_tokenize(text, language='portuguese') #Transforma o texto em tokens
   text_sem_stopwords = [token for token in tokenized if token not in stopwords] # Deixando somente o que nao é stopword no texto
   return text_sem_stopwords

df_va['tokenized_content'] = df_va['content'].apply(tokenized_cp)

In [19]:
df_va['tokenized_content'].sample(5)

1215    [senhores, prefeitura, municipal, nestes, mome...
1809    [senhor, prefeito, belém, história, rica, incl...
34      [outubro, srs, prefeitura, municipal, principi...
1056    [rio, janeiro, prefeitura, municipal, boa, tar...
562     [outubro, prefeitura, municipal, presente, est...
Name: tokenized_content, dtype: object

In [20]:
df_va.dtypes

file_name            object
content              object
nota                  int64
tokenized_content    object
dtype: object

In [21]:
# Contando o número de tokens SEM STOPWORDS (diferente do typestokenttr.ipynb) para cada texto: usarei isso para o gráfico 2, que demonstra o número mínimo e número máximo de tokens

# Contando tokens sem considerar a nota
df_va['num_tokens'] = df_va['tokenized_content'].apply(len)

# Agrupando de acordo com cada nota
qtde_tokens_nota = df_va.groupby('nota')['num_tokens'].sum()

In [22]:
df_va.tail()

,file_name,content,nota,tokenized_content,num_tokens
2288,20152t4p936n5r.docx,belem de outubro de senhores prefeitura munici...,5,"[belem, outubro, senhores, prefeitura, municip...",100
2289,20152t4p93r.docx,pará de outubro de ao senhor prefeito municipa...,5,"[pará, outubro, senhor, prefeito, municipal, p...",107
2290,20152t4p953n5r.docx,pilas de outubro de prefeitura municipal de be...,5,"[pilas, outubro, prefeitura, municipal, belém,...",94
2291,20152t4p96r.docx,para a prefeitura municipal de um morador de b...,5,"[prefeitura, municipal, morador, belém, senhor...",99
2292,20152t4p997n5r.docx,cidade do belém dia de outubro de exelentíssim...,5,"[cidade, belém, dia, outubro, exelentíssima, p...",145


In [23]:
# Agrupando a contagem de tokens de acordo com cada nota
nota_token_counts = (
    df_va.groupby('nota')['tokenized_content']
    .apply(lambda texts: Counter([token for text in texts for token in text]))
)

# Convertendo cada token e contagem em um dataframe
df_frequency = nota_token_counts.reset_index()
df_frequency.columns = ['nota', 'token', 'token_frequency']
sorted_tokens = df_frequency.sort_values('token_frequency', ascending=False)
sorted_tokens

,nota,token,token_frequency
44443,3,azulejos,2531.0
29648,2,azulejos,2173.0
44465,3,cidade,1838.0
59238,4,azulejos,1810.0
29670,2,cidade,1383.0
...,...,...,...
87640,5,vendeuse,NaN
87642,5,comunico,NaN
87644,5,roubáos,NaN
87645,5,impeder,NaN


In [24]:
df_words = pd.DataFrame()

In [41]:
# Agrupando a contagem de tokens por nota de forma a mostrar os 15 mais comuns para cada nota ( 15 * 6 = 90 linhas portanto )
top_tokens_per_grade = (
    sorted_tokens.groupby('nota')
    .head(20)  # Select the top 15 tokens per grade
    .reset_index(drop=True)
)
df_words = top_tokens_per_grade

In [42]:
df_words

,nota,token,token_frequency
0,3,azulejos,2531.0
1,2,azulejos,2173.0
2,3,cidade,1838.0
3,4,azulejos,1810.0
4,2,cidade,1383.0
...,...,...,...
115,0,fazer,8.0
116,0,problema,8.0
117,0,ano,8.0
118,0,parte,8.0


In [21]:
df_words_sorted = (df_words.sort_values(by=['nota', 'token_frequency'], ascending=[True, False]))
df_words_0 = df_words_sorted.loc[df_words_sorted['nota'] == 0, :]
df_words_1 = df_words_sorted.loc[df_words_sorted['nota'] == 1, :]
df_words_2 = df_words_sorted.loc[df_words_sorted['nota'] == 2, :]
df_words_3 = df_words_sorted.loc[df_words_sorted['nota'] == 3, :]
df_words_4 = df_words_sorted.loc[df_words_sorted['nota'] == 4, :]
df_words_5 = df_words_sorted.loc[df_words_sorted['nota'] == 5, :]
df_words_0 

,nota,token,token_frequency
100,0,azulejos,49.0
101,0,cidade,21.0
102,0,prefeitura,18.0
103,0,valiosos,13.0
104,0,casarões,13.0
105,0,situação,12.0
106,0,municipal,11.0
107,0,capital,10.0
108,0,pessoas,10.0
109,0,xix,10.0


In [26]:
# NOTA ZERO
import plotly.graph_objects as go
import pandas as pd


# Pivot the data for heatmap
heatmap_data = df_words_0.pivot(index='token', columns='nota', values='token_frequency').fillna(0)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    #colorbar=dict(title="Frequência"),  # Colorbar title
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "Token: %{y}<br>" + #Exibir título pelo Streamlit, q aí fica título pra cada coluna
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
   # title="20 Tokens Mais Frequentes", >> Por título no streamlit
    height=500,  # Adjust height to make the heatmaps narrow
    width=300,  # Adjust width for each heatmap ....  * len(grades)
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)
#fig.update_yaxes(title="Tokens", automargin=True)
fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()

In [27]:
# NOTA UM
import plotly.graph_objects as go
import pandas as pd


# Pivot the data for heatmap
heatmap_data = df_words_1.pivot(index='token', columns='nota', values='token_frequency').fillna(0)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    #colorbar=dict(title="Frequência"),  # Colorbar title
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "Token: %{y}<br>" + #Exibir título pelo Streamlit, q aí fica título pra cada coluna
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
   # title="20 Tokens Mais Frequentes", >> Por título no streamlit
    height=500,  # Adjust height to make the heatmaps narrow
    width=300,  # Adjust width for each heatmap ....  * len(grades)
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)
#fig.update_yaxes(title="Tokens", automargin=True)
fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()

In [28]:
# NOTA DOIS
import plotly.graph_objects as go
import pandas as pd


# Pivot the data for heatmap
heatmap_data = df_words_2.pivot(index='token', columns='nota', values='token_frequency').fillna(0)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    #colorbar=dict(title="Frequência"),  # Colorbar title
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "Token: %{y}<br>" + #Exibir título pelo Streamlit, q aí fica título pra cada coluna
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
   # title="20 Tokens Mais Frequentes", >> Por título no streamlit
    height=500,  # Adjust height to make the heatmaps narrow
    width=300,  # Adjust width for each heatmap ....  * len(grades)
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)
#fig.update_yaxes(title="Tokens", automargin=True)
fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()

In [29]:
# NOTA TRÊS
import plotly.graph_objects as go
import pandas as pd


# Pivot the data for heatmap
heatmap_data = df_words_3.pivot(index='token', columns='nota', values='token_frequency').fillna(0)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    #colorbar=dict(title="Frequência"),  # Colorbar title
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "Token: %{y}<br>" + #Exibir título pelo Streamlit, q aí fica título pra cada coluna
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
   # title="20 Tokens Mais Frequentes", >> Por título no streamlit
    height=500,  # Adjust height to make the heatmaps narrow
    width=300,  # Adjust width for each heatmap ....  * len(grades)
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)
#fig.update_yaxes(title="Tokens", automargin=True)
fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()

In [30]:
# NOTA QUATRO
import plotly.graph_objects as go
import pandas as pd


# Pivot the data for heatmap
heatmap_data = df_words_4.pivot(index='token', columns='nota', values='token_frequency').fillna(0)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    #colorbar=dict(title="Frequência"),  # Colorbar title
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "Token: %{y}<br>" + #Exibir título pelo Streamlit, q aí fica título pra cada coluna
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
   # title="20 Tokens Mais Frequentes", >> Por título no streamlit
    height=500,  # Adjust height to make the heatmaps narrow
    width=300,  # Adjust width for each heatmap ....  * len(grades)
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)
#fig.update_yaxes(title="Tokens", automargin=True)
fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()

In [31]:
# NOTA CINCO
import plotly.graph_objects as go
import pandas as pd


# Pivot the data for heatmap
heatmap_data = df_words_5.pivot(index='token', columns='nota', values='token_frequency').fillna(0)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    #colorbar=dict(title="Frequência"),  # Colorbar title
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "Token: %{y}<br>" + #Exibir título pelo Streamlit, q aí fica título pra cada coluna
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
   # title="20 Tokens Mais Frequentes", >> Por título no streamlit
    height=500,  # Adjust height to make the heatmaps narrow
    width=300,  # Adjust width for each heatmap ....  * len(grades)
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)
#fig.update_yaxes(title="Tokens", automargin=True)
fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()

In [27]:
df_words

,nota,token,token_frequency
0,3,azulejos,2531.0
1,2,azulejos,2173.0
2,3,cidade,1838.0
3,4,azulejos,1810.0
4,2,cidade,1383.0
...,...,...,...
115,0,fazer,8.0
116,0,problema,8.0
117,0,ano,8.0
118,0,parte,8.0


In [33]:
df_words_sorted = (df_words.sort_values(by=['nota', 'token_frequency'], ascending=[True, False]))
df_words_sorted

,nota,token,token_frequency
250,0,azulejos,49.0
251,0,cidade,21.0
252,0,prefeitura,18.0
253,0,valiosos,13.0
254,0,casarões,13.0
...,...,...,...
221,5,cidades,83.0
222,5,pessoas,82.0
225,5,artístico,79.0
230,5,roubados,77.0


In [37]:
df_words_sorted = (df_words.sort_values(by=['nota', 'token_frequency'], ascending=[False, False]))
df_words

,nota,token,token_frequency
0,3,azulejos,2531.0
1,2,azulejos,2173.0
2,3,cidade,1838.0
3,4,azulejos,1810.0
4,2,cidade,1383.0
...,...,...,...
295,0,patrimônio,5.0
296,0,departamento,5.0
297,0,construção,5.0
298,0,devem,5.0


In [55]:
import plotly.graph_objects as go ### ESSE AQUI QUE VOU USAR NO STREAMLIT ###
import pandas as pd

# Pivot the data for heatmap
heatmap_data = df_words_sorted.pivot(index='token', columns='nota', values='token_frequency').fillna(0).head(10)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    #colorbar=dict(title="Frequência"),  # Colorbar title
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "Token: %{y}<br>" + #Exibir título pelo Streamlit, q aí fica título pra cada coluna
        "Nota: %{x}<br>" +
        "Frequência: %{z}<extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
   # title="20 Tokens Mais Frequentes", >> Por título no streamlit
   # height=500,  # Adjust height to make the heatmaps narrow
  #  width=300,  # Adjust width for each heatmap ....  * len(grades)
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white")  # White font for all text
)
fig.update_yaxes(title="Tokens", automargin=True)
fig.update_xaxes(title="Frequência", showticklabels=False)

# Show the figure
fig.show()

In [54]:
# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  # Frequency matrix
    x=heatmap_data.columns,  # Grades (x-axis)
    y=heatmap_data.index,    # Tokens (y-axis)
    colorscale='Reds',       # Heatmap color scale
    colorbar=dict(title="Frequência", titlefont=dict(size=14), tickfont=dict(size=12)),  # Legend title and font size
    hoverongaps=False,       # Ensure no gaps show on hover
    hovertemplate=(
        "<span style='font-size:14px'><b>Token</b>: %{y}<br>" +  # Tooltip with larger font
        "<b>Nota</b>: %{x}<br>" +
        "<b>Frequência</b>: %{z}</span><extra></extra>"
    )
))

# Update layout for better visualization
fig.update_layout(
    template="plotly_white",
    plot_bgcolor="rgba(0, 0, 0, 0)",  # Transparent plot background
    paper_bgcolor="rgba(0, 0, 0, 0)",  # Transparent outer background
    font=dict(color="white", size=14),  # Update general font size
    height=800,  # Adjust height
    width=1000,   # Keep width consistent
)

# Update y-axis to hide tokens
fig.update_yaxes(
    title="",  # No y-axis title
    showticklabels=False,  # Hide tokens from y-axis
)

# Update x-axis for better styling
fig.update_xaxes(title="", showticklabels=False)

# Show the figure
fig.show()

